<a href="https://colab.research.google.com/github/mistralai/cookbook/blob/main/third_party/LlamaIndex/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation

# RAG Pipeline with LlamaIndex

In this notebook we will look into building RAG with LlamaIndex using `MistralAI LLM and Embedding Model`. Additionally, we will look into using Index as Retreiver.

1. Basic RAG pipeline.
2. Index as Retriever.

In [ ]:
!pip install llama-index 
!pip install llama-index-embeddings-mistralai
!pip install llama-index-llms-mistralai

## Setup API Keys

In [1]:
import os
os.environ['MISTRAL_API_KEY'] = '<YOUR MISTRALAI API KEY>'

## Basic RAG pipeline

Following are the steps involved in Builiding a basic RAG pipeline.

1. Setup LLM and Embedding Model
2. Download Data
3. Load Data
4. Create Nodes
5. Create Index
6. Create Query Engine
7. Querying

Query Engine combines `Retrieval` and `Response Synthesis` modules to generate response for the given query.

### Setup LLM and Embedding Model

In [2]:
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.mistralai import MistralAIEmbedding

In [3]:
llm = MistralAI(model='mistral-large')
embed_model = MistralAIEmbedding()

In [4]:
from llama_index.core import Settings
Settings.llm = llm
Settings.embed_model = embed_model

### Download Data

We will use `Uber 2021 10K SEC Filings` for the demonstration.

In [5]:
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O './uber_2021.pdf'

--2024-03-29 13:12:05--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘./uber_2021.pdf’

./uber_2021.pdf     100%[===================>]   1.79M  --.-KB/s    in 0.1s    

2024-03-29 13:12:05 (13.7 MB/s) - ‘./uber_2021.pdf’ saved [1880483/1880483]



### Load Data

In [6]:
from llama_index.core import SimpleDirectoryReader

In [7]:
documents = SimpleDirectoryReader(input_files=["./uber_2021.pdf"]).load_data()

### Create Nodes

In [8]:
from llama_index.core.node_parser import TokenTextSplitter

In [9]:
splitter = TokenTextSplitter(
    chunk_size=512,
    chunk_overlap=0,
)

nodes = splitter.get_nodes_from_documents(documents)

### Create Index

In [10]:
from llama_index.core import VectorStoreIndex

In [11]:
index = VectorStoreIndex(nodes)

### Create Query Engine

In [12]:
query_engine = index.as_query_engine(similarity_top_k=2)

### Querying

In [13]:
response = query_engine.query("What is the revenue of Uber in 2021?")

In [14]:
print(response)

The total revenue for Uber in 2021 was $17,455 million. This includes revenue from various offerings such as Mobility, Delivery, Freight, and All Other revenue streams. The Mobility revenue was $6,953 million, Delivery revenue was $8,362 million, Freight revenue was $2,132 million, and All Other revenue was $8 million.


## Index as Retriever

We can make use of created index as a `Retriever`. Retriever helps you to retrieve relevant chunks/ nodes for the given user query.

### Create Retriever

In [15]:
retriever = index.as_retriever(similarity_top_k = 2)

### Retrieve relevant nodes for a Query

In [16]:
retrieved_nodes = retriever.retrieve("What is the revenue of Uber in 2021?")

In [17]:
from llama_index.core.response.notebook_utils import display_source_node

for node in retrieved_nodes:
    display_source_node(node, source_length=1000)

**Node ID:** 96264fe5-bc88-4cf8-8905-a9691c39a5c9<br>**Similarity:** 0.8679960824403077<br>**Text:** Note 2 – RevenueThe
 following  tables  present  our  revenues  disaggregated  by  offering  and  geographical  region.  Revenue  by  geographical  region  is  based  on  where  thetransaction
 occurred. This level of disaggregation takes into consideration how the nature, amount, timing, and uncertainty of revenue and cash flows are affectedby economic factors. Revenue 
is presented in the following tables for the years ended December 31, 2019, 2020 and 2021, respectively (in millions):Year Ended December 31,
2019
2020 2021 Mobility revenue 
$ 10,707 $ 6,089 $ 6,953 Delivery revenue 
1,401 3,904 8,362 Freight revenue
731 1,011 2,132 All Other revenue
161 135 8 Total revenue
$ 13,000 $ 11,139 $ 17,455  We
 offer subscription memberships to end-users including Uber One, Uber Pass, Rides Pass, and Eats Pass (“Subscription”). We recognize Subscriptionfees
 ratably over the life of the pass. We allocate Subscription fees earned to Mobility and Delivery revenue on a proportional basis, b...<br>

**Node ID:** 653f0be9-ecfc-4fac-9488-afbd65f44ef2<br>**Similarity:** 0.8590901940430307<br>**Text:** COVID-19.
Revenue
 was $17.5 billion, or up 57% year-over-year, reflecting the overall growth in our Delivery business and an increase in Freight revenue attributable tothe
 acquisition of Transplace in the fourth quarter of 2021 as well as growth in the number of shippers and carriers on the network combined with an increase involumes with our top shippers.
Net
 loss attributable to Uber Technologies, Inc. was $496 million, a 93% improvement year-over-year, driven by a $1.6 billion pre-tax gain on the sale of ourATG
 Business to Aurora, a $1.6 billion pre-tax  net benefit relating to Uber’s equity investments, as  well as reductions in our fixed cost structure and increasedvariable cost effi
ciencies. Net loss attributable to Uber Technologies, Inc. also included $1.2 billion of stock-based compensation expense.Adjusted
 EBITDA loss was $774 million, improving $1.8 billion from 2020 with Mobility Adjusted EBITDA profit of $1.6 billion. Additionally, DeliveryAdjusted
 EBITDA loss of...<br>